In [1]:
import pandas as pd
import numpy as np
import folium
from geopy.distance import geodesic
from sklearn.preprocessing import StandardScaler

# Laad de gebruikersprofielen en routes
user_profiles_path = '../Data/Loopdata/kleinebestanden/run_gebruikersprofielen.json'
routes_path = '../Data/Loopdata/kleinebestanden/routes_volledig.json'
user_profiles_df = pd.read_json(user_profiles_path, lines=True)
routes_df = pd.read_json(routes_path, lines=True)

# Preprocessing en normalisatie
scaler = StandardScaler()
routes_df['mean_altitude'] = routes_df['altitude'].apply(np.mean)
user_profiles_df[['most_common_lat', 'most_common_lon']] = pd.DataFrame(user_profiles_df['average_location'].tolist(), index=user_profiles_df.index)
user_profiles_df[['average_speeds', 'average_altitude_changes', 'average_distances', 'average_condition_value']] = scaler.fit_transform(user_profiles_df[['average_speeds', 'average_altitude_changes', 'average_distances', 'average_condition_value']])
routes_df[['altitude_change', 'distance', 'mean_altitude', 'average_condition_value']] = scaler.fit_transform(routes_df[['altitude_change', 'distance', 'mean_altitude', 'average_condition_value']])

# Bepaal gewichten voor gebruikerskenmerken
gewichten = {row['userId']: {'average_altitude_change': row['weight_std_altitude_change'], 
                            'average_distance': row['weight_std_distance'], 
                            'average_condition_value': row['weight_std_condition_value'],
                            'distance_to_user': 1.0} for index, row in user_profiles_df.iterrows()}

# Functie om de afstand tussen twee punten te berekenen
def calculate_distance(point1, point2):
    return geodesic(point1, point2).kilometers

# Functie om een aansluitend route segment te vinden
def find_next_segment(current_location, remaining_segments, user_profile_weights):
    closest_segment = None
    min_distance = float('inf')
    
    for index, segment in remaining_segments.iterrows():
        segment_start_location = (segment['latitude'][0], segment['longitude'][0])
        distance_to_segment = calculate_distance(current_location, segment_start_location)

        if distance_to_segment < min_distance:
            min_distance = distance_to_segment
            closest_segment = segment
    
    return closest_segment

# Functie voor het genereren van nieuwe routes
def generate_new_route(user_id, user_profiles_df, routes_df, max_distance=10):
    user_profile = user_profiles_df[user_profiles_df['userId'] == user_id].iloc[0]
    user_location = (user_profile['most_common_lat'], user_profile['most_common_lon'])
    
    new_route_segments = []
    current_location = user_location
    remaining_distance = max_distance

    while remaining_distance > 0:
        remaining_segments = routes_df[routes_df['distance'] <= remaining_distance]
        next_segment = find_next_segment(current_location, remaining_segments, gewichten[user_id])

        if next_segment is not None:
            new_route_segments.append(next_segment)
            current_location = (next_segment['latitude'][-1], next_segment['longitude'][-1])
            remaining_distance -= next_segment['distance']
        else:
            break

    return pd.concat(new_route_segments)

# Test de functie voor een specifieke gebruiker
sample_user_id = 367324
new_route = generate_new_route(sample_user_id, user_profiles_df, routes_df)

# Zoek het gebruikersprofiel voor visualisatie
user_profile = user_profiles_df[user_profiles_df['userId'] == sample_user_id].iloc[0]

# Visualiseer de nieuwe route
map_obj = folium.Map(location=[user_profile['most_common_lat'], user_profile['most_common_lon']], zoom_start=12)
popup_text = f"UserID: {sample_user_id}<br>Average Speed: {user_profile['average_speeds']}<br>Average Altitude Change: {user_profile['average_altitude_changes']}<br>Average Distance: {user_profile['average_distances']}<br>Average Condition Value: {user_profile['average_condition_value']}"
folium.Marker(location=[user_profile['most_common_lat'], user_profile['most_common_lon']], popup=popup_text).add_to(map_obj)

for _, segment in new_route.iterrows():
    segment_coords = list(zip(segment['latitude'], segment['longitude']))
    folium.PolyLine(segment_coords, color='blue', weight=2.5, opacity=1).add_to(map_obj)

map_obj


In [11]:
import pandas as pd
import numpy as np
import folium
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist

# Laad de gebruikersprofielen en routes
user_profiles_df = pd.read_json(user_profiles_path, lines=True)
routes_df = pd.read_json(routes_path, lines=True)

# Preprocessing en normalisatie
scaler = StandardScaler()
routes_df['mean_altitude'] = routes_df['altitude'].apply(np.mean)
user_profiles_df[['most_common_lat', 'most_common_lon']] = pd.DataFrame(user_profiles_df['locations'].apply(lambda x: np.mean(x, axis=0)).tolist(), index=user_profiles_df.index)
user_profiles_df[['average_speeds', 'average_altitude_changes', 'average_distances', 'average_condition_value']] = scaler.fit_transform(user_profiles_df[['speeds', 'altitude_changes', 'distances', 'average_condition_value']])
routes_df[['altitude_change', 'distance', 'mean_altitude', 'average_condition_value']] = scaler.fit_transform(routes_df[['altitude_change', 'distance', 'mean_altitude', 'average_condition_value']])

# Bepaal gewichten voor gebruikerskenmerken
gewichten = user_profiles_df.set_index('userId')[['weight_std_altitude_change', 'weight_std_distance', 'weight_std_condition_value']].to_dict('index')

# Vectorized functie voor afstandsberekening
def calculate_distances(points1, points2):
    return cdist(points1, points2, metric='euclidean')

# Functie om een aansluitend route segment te vinden
def find_next_segment(current_location, remaining_segments):
    segment_start_locations = remaining_segments.apply(lambda x: (x['latitude'][0], x['longitude'][0]), axis=1).tolist()
    distances = calculate_distances([current_location], segment_start_locations)
    min_index = np.argmin(distances)
    return remaining_segments.iloc[min_index]

# Functie voor het genereren van nieuwe routes
def generate_new_route(user_id, max_distance=10):
    user_profile = user_profiles_df[user_profiles_df['userId'] == user_id].iloc[0]
    user_location = (user_profile['most_common_lat'], user_profile['most_common_lon'])

    new_route_segments = []
    current_location = user_location
    remaining_distance = max_distance

    while remaining_distance > 0:
        remaining_segments = routes_df[routes_df['distance'] <= remaining_distance]
        if remaining_segments.empty:
            break
        next_segment = find_next_segment(current_location, remaining_segments)
        new_route_segments.append(next_segment)
        current_location = (next_segment['latitude'][-1], next_segment['longitude'][-1])
        remaining_distance -= next_segment['distance']

    return pd.concat(new_route_segments, ignore_index=True)

# Test de functie voor een specifieke gebruiker
sample_user_id = 367324
new_route = generate_new_route(sample_user_id)

# Visualiseer de nieuwe route
map_obj = folium.Map(location=[user_profiles_df.loc[user_profiles_df['userId'] == sample_user_id, 'most_common_lat'].iloc[0], user_profiles_df.loc[user_profiles_df['userId'] == sample_user_id, 'most_common_lon'].iloc[0]], zoom_start=12)

for _, segment in new_route.iterrows():
    segment_coords = list(zip(segment['latitude'], segment['longitude']))
    folium.PolyLine(segment_coords, color='blue', weight=2.5, opacity=1).add_to(map_obj)

map_obj


ValueError: setting an array element with a sequence.

In [5]:
import pandas as pd
import numpy as np
import json
from sklearn.preprocessing import StandardScaler
from geopy.distance import geodesic
import folium
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

# Pad naar de bestanden
user_profiles_path = '../Data/Loopdata/kleinebestanden/run_gebruikersprofielen.json'
routes_path = '../Data/Loopdata/kleinebestanden/routes_volledig.json'

# Laad de gebruikersprofielen en routes
user_profiles_df = pd.read_json(user_profiles_path, lines=True)
routes_df = pd.read_json(routes_path, lines=True)

# Preprocessing van de routegegevens
routes_df['mean_altitude'] = routes_df['altitude'].apply(np.mean)
routes_df['mean_latitude'] = routes_df['latitude'].apply(np.mean)
routes_df['mean_longitude'] = routes_df['longitude'].apply(np.mean)
user_profiles_df[['most_common_lat', 'most_common_lon']] = pd.DataFrame(user_profiles_df['average_location'].tolist(), index=user_profiles_df.index)

# Normaliseer de kenmerken met StandardScaler
scaler = StandardScaler()
user_profiles_df[['average_speeds', 'average_altitude_changes', 'average_distances', 'average_condition_value']] = scaler.fit_transform(user_profiles_df[['average_speeds', 'average_altitude_changes', 'average_distances', 'average_condition_value']])
routes_df[['altitude_change', 'distance', 'mean_altitude', 'average_condition_value']] = scaler.fit_transform(routes_df[['altitude_change', 'distance', 'mean_altitude', 'average_condition_value']])

# Bepaal gewichten voor gebruikerskenmerken
gewichten = {}
for index, row in user_profiles_df.iterrows():
    user_id = row['userId']
    gewichten[user_id] = {
        'average_altitude_change': row['weight_std_altitude_change'], 
        'average_distance': row['weight_std_distance'], 
        'average_condition_value': row['weight_std_condition_value'],
        'distance_to_user': 1.0}

# Definieer de functie voor het genereren van nieuwe routes
def generate_new_routes(user_id, user_profiles_df, routes_df, num_segments=5, max_distance=10):
    user_data = user_profiles_df[user_profiles_df['userId'] == user_id].iloc[0]
    user_location = (user_data['most_common_lat'], user_data['most_common_lon'])

    # Selecteer segmenten die voldoen aan gebruikersvoorkeuren
    # Dit is een gesimplificeerde benadering; een meer geavanceerde methode zou nodig zijn voor een realistische implementatie
    candidate_segments = routes_df[routes_df['distance'] < max_distance]

    # Genereer een nieuwe route door segmenten willekeurig te combineren
    new_route = candidate_segments.sample(n=num_segments)
    return new_route

# Test de functie voor een specifieke gebruiker
sample_user_id = 367324
new_route = generate_new_routes(sample_user_id, user_profiles_df, routes_df)

# Visualiseer de nieuwe route
map_obj = folium.Map(location=user_profiles_df.loc[user_profiles_df['userId'] == sample_user_id, ['most_common_lat', 'most_common_lon']].values[0], zoom_start=12)

for _, route_data in new_route.iterrows():
    route_coords = list(zip(route_data['latitude'], route_data['longitude']))
    folium.PolyLine(route_coords, color='blue', weight=2.5, opacity=1).add_to(map_obj)

map_obj

In [ ]:
import json
from geopy.distance import geodesic
import itertools

def calculate_route_distance(coords):
    """ Bereken de afstand van een route op basis van coördinaten. """
    total_distance = 0
    for i in range(1, len(coords)):
        total_distance += geodesic(coords[i - 1], coords[i]).kilometers
    return total_distance

def are_points_close(point1, point2, max_distance=0.1):
    """ Controleer of twee punten binnen een bepaalde afstand van elkaar liggen. """
    return geodesic(point1, point2).kilometers <= max_distance

def find_partial_connecting_routes(routes, target_distance=10, distance_tolerance=2, proximity_threshold=0.1):
    """ Vind aansluitende routes of delen van routes die samen een gespecificeerde afstand benaderen. """
    connected_routes = []

    # Alle combinaties van routes overwegen
    for route1, route2 in itertools.permutations(routes, 2):
        for split_index in range(1, len(route2['coords'])):
            # Deel van route2 nemen
            partial_route2 = route2['coords'][:split_index]
            # Controleer of de punten dichtbij elkaar liggen
            if are_points_close(route1['coords'][-1], partial_route2[0], proximity_threshold):
                combined_coords = route1['coords'] + partial_route2[1:]  # Voorkomen van duplicatie van het aansluitpunt
                combined_distance = calculate_route_distance(combined_coords)
                if abs(combined_distance - target_distance) <= distance_tolerance:  # Vergrote tolerantie
                    connected_routes.append({
                        'combined_coords': combined_coords,
                        'combined_distance': combined_distance
                    })

    return connected_routes

# Bestand inlezen en routes extraheren
file_path = 'jouw_bestandspad_hier.json'
extracted_routes = []
with open(file_path, 'r') as file:
    for line in file:
        route = json.loads(line)
        coords = list(zip(route['latitude'], route['longitude']))
        extracted_routes.append({'coords': coords, 'distance': route['distance']})

# Zoeken naar aansluitende routes met een totale afstand van ongeveer 10 km met een vergrote tolerantie
partial_connected_routes = find_partial_connecting_routes(extracted_routes)

# Resultaten weergeven
for route in partial_connected_routes:
    print("Gecombineerde Afstand: {:.2f} km".format(route['combined_distance']))
    print("Coördinaten:", route['combined_coords'])


In [ ]:
import json
from itertools import combinations

# Function to check if two routes partially overlap
def check_partial_overlap(route1, route2, segment_length=10):
    """Check if a segment of route1 overlaps with any segment of route2. 
    Segment length can be adjusted."""
    for i in range(len(route1) - segment_length + 1):
        segment1 = set(route1[i:i + segment_length])
        for j in range(len(route2) - segment_length + 1):
            segment2 = set(route2[j:j + segment_length])
            if segment1 == segment2:
                return True
    return False

# Path to the file containing the routes
file_path = '/mnt/data/100_routes.json'

# Reading the entire file and parsing each line as a JSON object
routes = []
with open(file_path, 'r') as file:
    for line in file:
        routes.append(json.loads(line))

# Extracting latitude and longitude coordinates for each route
route_coordinates = []
for route in routes:
    latitudes = route.get('latitude', [])
    longitudes = route.get('longitude', [])
    route_coordinates.append(list(zip(latitudes, longitudes)))

# Check for partial overlaps between all pairs of routes
overlaps = {}
for i, j in combinations(range(len(route_coordinates)), 2):
    if check_partial_overlap(route_coordinates[i], route_coordinates[j]):
        overlaps[(i, j)] = True

# Displaying the count and the indices of the overlapping route pairs
len(overlaps), overlaps.keys()


In [1]:
import json
import random
import folium
import haversine

# Functie om de afstand tussen twee coördinaten te berekenen
def bereken_afstand(coord1, coord2):
    return haversine.haversine(coord1, coord2, unit=haversine.Unit.KILOMETERS)

# Stap 1: Data Voorbereiding
def laad_data(file_path):
    routes = []
    with open(file_path, 'r') as file:
        for line in file:
            # Parse elke regel als een JSON-object
            route = json.loads(line.strip())
            routes.append(route)
    return routes

# Stap 2: Analyse van de Route Structuur
def identificeer_route_segmenten(routes):
    segmenten = []
    for route in routes:
        # Aanname: De structuur van de route bevat 'latitude' en 'longitude' arrays
        latitudes = route.get('latitude', [])
        longitudes = route.get('longitude', [])
        # Verdeel de route in segmenten, hier eenvoudigweg door de route in tweeën te splitsen
        mid_index = len(latitudes) // 2
        segmenten.append((latitudes[:mid_index], longitudes[:mid_index]))
        segmenten.append((latitudes[mid_index:], longitudes[mid_index:]))
    return segmenten

# Stap 3: Route Generatie
def genereer_nieuwe_routes(segmenten, aantal_routes=10, max_afstand=10):
    nieuwe_routes = []
    for _ in range(aantal_routes):
        nieuwe_route = {'latitude': [], 'longitude': []}

        # Kies een willekeurig startsegment
        start_segment = random.choice(segmenten)
        nieuwe_route['latitude'] = start_segment[0]
        nieuwe_route['longitude'] = start_segment[1]

        # Voeg andere segmenten toe die dichtbij zijn
        for __ in range(random.randint(1, 4)):  # Aantal extra segmenten
            laatste_punt = (nieuwe_route['latitude'][-1], nieuwe_route['longitude'][-1])
            dichtbij_segmenten = [(seg, bereken_afstand(laatste_punt, (seg[0][0], seg[1][0]))) for seg in segmenten]
            dichtbij_segmenten = [seg for seg, dist in dichtbij_segmenten if dist < max_afstand]
            
            if dichtbij_segmenten:
                volgende_segment = random.choice(dichtbij_segmenten)
                nieuwe_route['latitude'] += volgende_segment[0]
                nieuwe_route['longitude'] += volgende_segment[1]

        nieuwe_routes.append(nieuwe_route)
    return nieuwe_routes
# Stap 4: Validatie en Verfijning (vereenvoudigde weergave)
def valideer_routes(routes):
    # Deze functie zou meer complexe logica moeten bevatten om routes te valideren
    # Voor dit voorbeeld doen we een eenvoudige check
    return [route for route in routes if len(route['latitude']) > 0 and len(route['longitude']) > 0]

# Functie om een route te visualiseren op een kaart met folium
def visualiseer_meerdere_routes(routes):
    # Maak een nieuwe kaart aan, gecentreerd op het eerste punt van de eerste route
    if routes:
        m = folium.Map(location=[routes[0]['latitude'][0], routes[0]['longitude'][0]], zoom_start=12)
    else:
        # Als er geen routes zijn, maak een standaardkaart
        m = folium.Map(location=[0, 0], zoom_start=2)

    # Voeg elke route toe als een lijn op de kaart
    for route in routes:
        folium.PolyLine(list(zip(route['latitude'], route['longitude'])), color=random_color(), weight=2.5, opacity=1).add_to(m)
    return m

# Functie om een willekeurige kleur te genereren voor de visualisatie
def random_color():
    import random
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))

# Voer de stappen uit met een voorbeeldbestand
file_path = '../Data/Loopdata/kleinebestanden/routes_volledig.json'  # Verander dit naar het werkelijke pad van uw bestand
routes = laad_data(file_path)
segmenten = identificeer_route_segmenten(routes)
nieuwe_routes = genereer_nieuwe_routes(segmenten, 10)
gevalideerde_routes = valideer_routes(nieuwe_routes)

# Visualiseer alle gegenereerde en gevalideerde routes
kaart = visualiseer_meerdere_routes(gevalideerde_routes)
kaart.save('../Data/Loopdata/kleinebestanden/visualized_multiple_routes.html')  # Sla de kaart op als een HTML-bestand

# De code retourneert het pad naar het opgeslagen HTML-bestand
'../Data/Loopdata/kleinebestanden/visualized_multiple_routes.html'


'../Data/Loopdata/kleinebestanden/visualized_multiple_routes.html'

In [9]:
import json
import random
import folium
import haversine

# Functie om de afstand tussen twee coördinaten te berekenen
def bereken_afstand(coord1, coord2):
    return haversine.haversine(coord1, coord2, unit=haversine.Unit.KILOMETERS)

def bereken_route_afstand(route):
    totale_afstand = 0
    for i in range(1, len(route['latitude'])):
        punt1 = (route['latitude'][i - 1], route['longitude'][i - 1])
        punt2 = (route['latitude'][i], route['longitude'][i])
        totale_afstand += bereken_afstand(punt1, punt2)
    return totale_afstand

# Stap 1: Data Voorbereiding
def laad_data(file_path):
    routes = []
    with open(file_path, 'r') as file:
        for line in file:
            # Parse elke regel als een JSON-object
            route = json.loads(line.strip())
            routes.append(route)
    return routes

# Stap 2: Analyse van de Route Structuur (aangepast)
def identificeer_route_segmenten(routes):
    segmenten = []
    for route_idx, route in enumerate(routes):
        latitudes = route.get('latitude', [])
        longitudes = route.get('longitude', [])
        mid_index = len(latitudes) // 2
        # Voeg route_idx toe aan elk segment
        segmenten.append(((latitudes[:mid_index], longitudes[:mid_index]), route_idx))
        segmenten.append(((latitudes[mid_index:], longitudes[mid_index:]), route_idx))
    return segmenten

# Stap 3: Route Generatie (aangepast)
def genereer_nieuwe_routes(segmenten, aantal_routes=10, max_afstand=1):
    nieuwe_routes = []
    for _ in range(aantal_routes):
        nieuwe_route = {'latitude': [], 'longitude': [], 'oorspronkelijke_route_ids': set()}
        gebruikte_route_ids = set()

        start_segment, route_id = random.choice(segmenten)
        nieuwe_route['latitude'] = start_segment[0]
        nieuwe_route['longitude'] = start_segment[1]
        nieuwe_route['oorspronkelijke_route_ids'].add(route_id)
        gebruikte_route_ids.add(route_id)

        for __ in range(random.randint(1, 4)):
            laatste_punt = (nieuwe_route['latitude'][-1], nieuwe_route['longitude'][-1])
            dichtbij_segmenten = [
                (seg, bereken_afstand(laatste_punt, (seg[0][0], seg[0][1])), rid) 
                for seg, rid in segmenten if rid not in gebruikte_route_ids
            ]
            dichtbij_segmenten = [seg for seg in dichtbij_segmenten if seg[1] < max_afstand]
            
            if dichtbij_segmenten:
                volgende_segment, _, rid = random.choice(dichtbij_segmenten)
                nieuwe_route['latitude'] += volgende_segment[0]
                nieuwe_route['longitude'] += volgende_segment[1]
                nieuwe_route['oorspronkelijke_route_ids'].add(rid)
                gebruikte_route_ids.add(rid)

        nieuwe_route['oorspronkelijke_route_ids'] = list(nieuwe_route['oorspronkelijke_route_ids'])
        nieuwe_routes.append(nieuwe_route)
    return nieuwe_routes

# Stap 4: Validatie en Verfijning (vereenvoudigde weergave)
def valideer_routes(routes):
    # Deze functie zou meer complexe logica moeten bevatten om routes te valideren
    # Voor dit voorbeeld doen we een eenvoudige check
    return [route for route in routes if len(route['latitude']) > 0 and len(route['longitude']) > 0]

# Functie om een route te visualiseren op een kaart met folium
def visualiseer_meerdere_routes(routes):
    if routes:
        m = folium.Map(location=[routes[0]['latitude'][0], routes[0]['longitude'][0]], zoom_start=12)
    else:
        m = folium.Map(location=[0, 0], zoom_start=2)

    for route in routes:
        afstand = bereken_route_afstand(route)
        oorsprong_info = ', '.join(map(str, route['oorspronkelijke_route_ids']))
        route_info = f"Afstand: {afstand:.2f} km, Oorsprong: {oorsprong_info}"
        folium.PolyLine(
            list(zip(route['latitude'], route['longitude'])), 
            color=random.choice(['#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#00FFFF', '#FF00FF']),  # Willekeurige kleur voor elke route
            weight=2.5, 
            opacity=1,
            popup=route_info
        ).add_to(m)
    return m


# Voer de stappen uit met een voorbeeldbestand
file_path = '../Data/Loopdata/kleinebestanden/routes_volledig.json'  # Verander dit naar het werkelijke pad van uw bestand
routes = laad_data(file_path)
segmenten = identificeer_route_segmenten(routes)
nieuwe_routes = genereer_nieuwe_routes(segmenten, 1)
gevalideerde_routes = valideer_routes(nieuwe_routes)

# Visualiseer alle gegenereerde en gevalideerde routes
kaart = visualiseer_meerdere_routes(gevalideerde_routes)
kaart.save('../Data/Loopdata/kleinebestanden/visualized_multiple_routes.html')  # Sla de kaart op als een HTML-bestand

# De code retourneert het pad naar het opgeslagen HTML-bestand
'../Data/Loopdata/kleinebestanden/visualized_multiple_routes.html'


'../Data/Loopdata/kleinebestanden/visualized_multiple_routes.html'

In [2]:
import json
import geopy.distance

# Functie om de afstand tussen twee coördinaten te berekenen
def calculate_distance(coord1, coord2):
    return geopy.distance.geodesic(coord1, coord2).km

# Functie om de totale afstand van een route te berekenen
def route_distance(route):
    total_distance = 0
    for i in range(len(route["latitude"]) - 1):
        coord1 = (route["latitude"][i], route["longitude"][i])
        coord2 = (route["latitude"][i + 1], route["longitude"][i + 1])
        total_distance += calculate_distance(coord1, coord2)
    return total_distance

# Functie om mogelijke segmentcombinaties te vinden die een totale afstand van ongeveer 10 km hebben
def find_route_combinations(routes, target_distance=10.0, tolerance=0.5):
    combinations = []
    for i, route1 in enumerate(routes):
        for j, route2 in enumerate(routes):
            if i != j:
                total_distance = route1["calculated_distance"] + route2["calculated_distance"]
                if abs(total_distance - target_distance) <= tolerance:
                    combinations.append((i, j, total_distance))
    return combinations

# Functie om te controleren of twee routes geografisch aansluiten
def are_routes_adjacent(route1, route2, max_distance=0.5):
    end_point_route1 = (route1["latitude"][-1], route1["longitude"][-1])
    start_point_route2 = (route2["latitude"][0], route2["longitude"][0])
    distance = calculate_distance(end_point_route1, start_point_route2)
    return distance <= max_distance

# Routes inlezen en totale afstand voor elke route berekenen
routes = []
with open('../Data/Loopdata/kleinebestanden/100_routes.json', 'r') as file:
    for line in file:
        route = json.loads(line)
        route["calculated_distance"] = route_distance(route)
        routes.append(route)

# Vind combinaties van routes die dicht bij 10 km zijn
route_combinations = find_route_combinations(routes)

# Analyseer de voorgestelde combinaties om haalbare route combinaties te vinden
feasible_combinations = []
for combo in route_combinations:
    route1_index, route2_index, total_distance = combo
    route1 = routes[route1_index]
    route2 = routes[route2_index]
    if are_routes_adjacent(route1, route2):
        combined_lat = route1["latitude"] + route2["latitude"]
        combined_lon = route1["longitude"] + route2["longitude"]
        feasible_combinations.append({
            "route1_index": route1_index, 
            "route2_index": route2_index, 
            "total_distance": total_distance,
            "latitude": combined_lat,
            "longitude": combined_lon
        })

# Voorbeeld: Gegevens van de eerste haalbare combinatie weergeven, inclusief coördinaten voor visualisatie
first_combination = feasible_combinations[0]
first_combination


{'route1_index': 21,
 'route2_index': 24,
 'total_distance': 10.110175177008713,
 'latitude': [51.907620364800096,
  51.90763561986387,
  51.90769881941378,
  51.90771893598139,
  51.90775481052697,
  51.907806443050504,
  51.9078307505697,
  51.907901829108596,
  51.90794935449958,
  51.90800744108856,
  51.90806468948722,
  51.908149011433125,
  51.90821430645883,
  51.90823366865516,
  51.90830164588988,
  51.90834724344313,
  51.9084489159286,
  51.90846827812493,
  51.90859677270055,
  51.90861881710589,
  51.90866290591657,
  51.90871956758201,
  51.90887102857232,
  51.90902148373425,
  51.90916271880269,
  51.90917914733291,
  51.90930537879467,
  51.90941761247814,
  51.90953068435192,
  51.909651132300496,
  51.909769820049405,
  51.90986570902169,
  51.909973081201315,
  51.910050781443715,
  51.91015756689012,
  51.9102449901402,
  51.910332161933184,
  51.91045646555722,
  51.9105782546103,
  51.91069459542632,
  51.91080599091947,
  51.910924427211285,
  51.91100950352847

In [6]:
import json
import geopy.distance
import folium

# Functie om de afstand tussen twee coördinaten te berekenen
def calculate_distance(coord1, coord2):
    return geopy.distance.geodesic(coord1, coord2).km

# Functie om de totale afstand van een route te berekenen
def route_distance(route):
    total_distance = 0
    for i in range(len(route["latitude"]) - 1):
        coord1 = (route["latitude"][i], route["longitude"][i])
        coord2 = (route["latitude"][i + 1], route["longitude"][i + 1])
        total_distance += calculate_distance(coord1, coord2)
    return total_distance

# Functie om mogelijke segmentcombinaties te vinden die een totale afstand van ongeveer 10 km hebben
def find_route_combinations(routes, target_distance=10.0, tolerance=0.5):
    combinations = []
    for i, route1 in enumerate(routes):
        for j, route2 in enumerate(routes):
            if i != j:
                total_distance = route1["calculated_distance"] + route2["calculated_distance"]
                if abs(total_distance - target_distance) <= tolerance:
                    combinations.append((i, j, total_distance))
    return combinations

# Functie om te controleren of twee routes geografisch aansluiten
def are_routes_adjacent(route1, route2, max_distance=0.5):
    end_point_route1 = (route1["latitude"][-1], route1["longitude"][-1])
    start_point_route2 = (route2["latitude"][0], route2["longitude"][0])
    distance = calculate_distance(end_point_route1, start_point_route2)
    return distance <= max_distance

# Routes inlezen en totale afstand voor elke route berekenen
routes = []
with open('../Data/Loopdata/kleinebestanden/100_routes.json', 'r') as file:
    for line in file:
        route = json.loads(line)
        route["calculated_distance"] = route_distance(route)
        routes.append(route)

# Vind combinaties van routes die dicht bij 10 km zijn
route_combinations = find_route_combinations(routes)
print(len(route_combinations))

# Analyseer de voorgestelde combinaties om haalbare route combinaties te vinden
feasible_combinations = []
for combo in route_combinations:
    route1_index, route2_index, total_distance = combo
    route1 = routes[route1_index]
    route2 = routes[route2_index]
    if are_routes_adjacent(route1, route2):
        combined_lat = route1["latitude"] + route2["latitude"]
        combined_lon = route1["longitude"] + route2["longitude"]
        feasible_combinations.append({
            "route1_index": route1_index, 
            "route2_index": route2_index, 
            "total_distance": total_distance,
            "latitude": combined_lat,
            "longitude": combined_lon
        })

# Functie om de gecombineerde route te visualiseren op een kaart met Folium
def visualize_routes(combinations):
    # Bepaal het centrum voor de initiële kaartweergave (gebruik het beginpunt van de eerste combinatie)
    if combinations:
        center_lat, center_lon = combinations[0]["latitude"][0], combinations[0]["longitude"][0]
    else:
        center_lat, center_lon = 0, 0  # Gebruik een standaardlocatie als er geen combinaties zijn

    m = folium.Map(location=[center_lat, center_lon], zoom_start=14)

    # Teken elke gecombineerde route
    for combination in combinations:
        route1_index = combination["route1_index"]
        route2_index = combination["route2_index"]
        popup_text = f"Gecombineerde route van Route {route1_index} en Route {route2_index}"
        folium.PolyLine(
            list(zip(combination["latitude"], combination["longitude"])),
            color="blue",
            weight=2.5,
            opacity=1,
            popup=folium.Popup(popup_text)
        ).add_to(m)

    return m

# Bestaande code om feasible_combinations te genereren...

# Visualiseer de eerste 5 haalbare combinaties (of minder als er minder dan 5 zijn)
map_visualization = visualize_routes(feasible_combinations[:5])

# Om de kaart te tonen (als je in een Jupyter notebook werkt):
map_visualization


196


In [2]:
import json
import geopy.distance
import folium
import itertools
# Functie om de afstand tussen twee coördinaten te berekenen
def calculate_distance(coord1, coord2):
    return geopy.distance.geodesic(coord1, coord2).km

# Functie om de totale afstand van een route te berekenen
def route_distance(route):
    total_distance = 0
    for i in range(len(route["latitude"]) - 1):
        coord1 = (route["latitude"][i], route["longitude"][i])
        coord2 = (route["latitude"][i + 1], route["longitude"][i + 1])
        total_distance += calculate_distance(coord1, coord2)
    return total_distance

# Functie om mogelijke segmentcombinaties te vinden die een totale afstand van ongeveer 10 km hebben
def find_route_combinations(routes, target_distance=10.0, tolerance=0.5):
    combinations = []
    for i, route1 in enumerate(routes):
        for j, route2 in enumerate(routes):
            if i != j:
                total_distance = route1["calculated_distance"] + route2["calculated_distance"]
                if abs(total_distance - target_distance) <= tolerance:
                    combinations.append((i, j, total_distance))
    return combinations

# Functie om te controleren of twee routes geografisch aansluiten
def are_routes_adjacent(route1, route2, max_distance=0.5):
    end_point_route1 = (route1["latitude"][-1], route1["longitude"][-1])
    start_point_route2 = (route2["latitude"][0], route2["longitude"][0])
    distance = calculate_distance(end_point_route1, start_point_route2)
    return distance <= max_distance

# Routes inlezen en totale afstand voor elke route berekenen
routes = []
with open('../Data/Loopdata/kleinebestanden/routes_volledig.json', 'r') as file:
    for line in file:
        route = json.loads(line)
        route["calculated_distance"] = route_distance(route)
        routes.append(route)

# Vind combinaties van routes die dicht bij 10 km zijn
route_combinations = find_route_combinations(routes)
# print(len(route_combinations))

# Analyseer de voorgestelde combinaties om haalbare route combinaties te vinden
# Aangepaste functie om een beperkt aantal haalbare combinaties te vinden
def find_feasible_combinations(routes, max_combinations=5):
    feasible_combinations = []
    for i, route1 in enumerate(routes):
        for j, route2 in enumerate(routes):
            if i != j:
                total_distance = route1["calculated_distance"] + route2["calculated_distance"]
                if abs(total_distance - 10.0) <= 0.5 and are_routes_adjacent(route1, route2):
                    combined_lat = route1["latitude"] + route2["latitude"]
                    combined_lon = route1["longitude"] + route2["longitude"]
                    feasible_combinations.append({
                        "route1_index": i, 
                        "route2_index": j, 
                        "total_distance": total_distance,
                        "latitude": combined_lat,
                        "longitude": combined_lon
                    })
                    if len(feasible_combinations) >= max_combinations:
                        return feasible_combinations
    return feasible_combinations

feasible_combinations = find_feasible_combinations(routes)
print(len(feasible_combinations))

# Functie om de gecombineerde route te visualiseren op een kaart met Folium
def visualize_routes(combinations):
    # Bepaal het centrum voor de initiële kaartweergave (gebruik het beginpunt van de eerste combinatie)
    if combinations:
        center_lat, center_lon = combinations[0]["latitude"][0], combinations[0]["longitude"][0]
    else:
        center_lat, center_lon = 0, 0  # Gebruik een standaardlocatie als er geen combinaties zijn

    m = folium.Map(location=[center_lat, center_lon], zoom_start=14)
    
    # Maak een lijst van verschillende kleuren
    colors = itertools.cycle(['blue', 'red', 'green', 'purple', 'orange'])

    # Teken elke gecombineerde route
    for combination, color in zip(combinations, colors):
        route1_index = combination["route1_index"]
        route2_index = combination["route2_index"]
        popup_text = f"Gecombineerde route van Route {route1_index} en Route {route2_index}"
        folium.PolyLine(
            list(zip(combination["latitude"], combination["longitude"])),
            color=color,  # Gebruik de volgende kleur in de lijst
            weight=2.5,
            opacity=1,
            popup=folium.Popup(popup_text)
        ).add_to(m)

    return m

# Visualiseer de eerste 5 haalbare combinaties (of minder als er minder dan 5 zijn)
map_visualization = visualize_routes(feasible_combinations[:20])

# Toon de kaart (als je in een Jupyter Notebook werkt)
map_visualization 


5


In [ ]:
import json
import geopy.distance
from itertools import combinations

# Functie om de afstand tussen twee coördinaten te berekenen
def calculate_distance(coord1, coord2):
    return geopy.distance.geodesic(coord1, coord2).km

# Functie om de totale afstand van een route te berekenen
def route_distance(route):
    total_distance = 0
    for i in range(len(route["latitude"]) - 1):
        coord1 = (route["latitude"][i], route["longitude"][i])
        coord2 = (route["latitude"][i + 1], route["longitude"][i + 1])
        total_distance += calculate_distance(coord1, coord2)
    return total_distance

# Routes inlezen en totale afstand voor elke route berekenen
routes = []
file_path = 'path_to_your_json_file'  # Verander dit naar het pad van je JSON-bestand
with open(file_path, 'r') as file:
    for line in file:
        route = json.loads(line)
        route["calculated_distance"] = route_distance(route)
        routes.append(route)

# Functie om combinaties van meerdere segmenten te vinden die een totale afstand van ongeveer 10 km hebben
def find_route_combinations(routes, target_distance=10.0, max_segments=3, tolerance=0.5):
    route_combinations = []
    for segment_count in range(2, max_segments + 1):
        for route_combo in combinations(routes, segment_count):
            total_distance = sum(route["calculated_distance"] for route in route_combo)
            if abs(total_distance - target_distance) <= tolerance:
                route_combinations.append(route_combo)
    return route_combinations

# Vind combinaties van routes die dicht bij 10 km zijn
route_combinations = find_route_combinations(routes)
print(f"Aantal gevonden combinaties: {len(route_combinations)}")
